In [1]:
!pip install scipy
!pip install sentence_transformers
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 16.5 MB/s eta 0:00:00


In [16]:
import torch
import torch.nn as nn
# import math
# from scipy.stats import special_ortho_group
# import matplotlib.pyplot as plt
# import random

# # for AE and Embedder
# import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
from torch.utils.data import DataLoader, TensorDataset


# from torchvision.datasets import MNIST
# from torchvision import transforms
# from torch.utils.data import Dataset, DataLoader, random_split
from cINN import *
from autoencoder_simple import AutoencoderSimple

ae_path = "/content/ae_100.pth"

In [18]:
def train_cinn(model, x_data, y_data, batch_size=32, epochs=1000, lr=0.001):
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)

    # Convert each element in x_data and y_data to a scalar
    x_data = [torch.Tensor(data_point).squeeze() for data_point in x_data]
    y_data = [torch.Tensor(data_point).squeeze() for data_point in y_data]

    data = TensorDataset(torch.stack(x_data), torch.stack(y_data))
    dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        for x_batch, y_batch in dataloader:
            optimizer.zero_grad()

            z, ljd = model(x_batch, y_batch)

            loss = torch.sum(0.5 * torch.sum(z**2, -1) - ljd) / batch_size
            loss.backward(retain_graph=True)

            optimizer.step()

        if (epoch+1) % (epochs//5) == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Create the training Data derived only form the Embeddig model an AE

In [5]:
emb_model = SentenceTransformer("intfloat/multilingual-e5-small")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [6]:
import torch

def get_best_device():
    """
    Returns the best available device for computation based on the system configuration.

    Priority:
    1. CUDA GPU if available
    2. MPS (Multi-Process Service) if available
    3. CPU as a fallback

    Returns:
        torch.device: The selected device for computation.
    """
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device('mps')
    else:
        return torch.device("cpu")


In [7]:
AE = AutoencoderSimple()
AE.load_state_dict(torch.load(ae_path, map_location=get_best_device()))

<All keys matched successfully>

In [8]:
import torch
from torchvision import datasets, transforms
from sentence_transformers import SentenceTransformer
from torch.utils.data import DataLoader

# Download MNIST dataset
mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=mnist_transform)

# Create DataLoader for batch processing
data_loader = DataLoader(mnist_dataset, batch_size=64, shuffle=True)

# Assuming you have a PyTorch model named ae and an embedding model named emb_model
# ae.encoder(x) should encode an image x
# emb_model(string) should embed a label

# Replace images with encoded latent space representation and replace labels with embeddings
encoded_dataset = []

for batch_imgs, batch_labels in data_loader:
    encoded_imgs = AE.encoder(batch_imgs)
    encoded_dataset.extend(zip(encoded_imgs, batch_labels))

# Convert labels to strings if not already
encoded_dataset = [(img, str(label)) for img, label in encoded_dataset]

# Embed labels efficiently using SentenceTransformer
embedded_labels = emb_model.encode(list(zip(*encoded_dataset))[1])
embedded_dataset = list(zip(list(zip(*encoded_dataset))[0], embedded_labels))

# You can use embedded_dataset for your further processing


100%|██████████| 9912422/9912422 [00:00<00:00, 118761279.61it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 34620089.69it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 31940677.88it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 16652560.11it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [9]:
print(len(embedded_dataset))
print(len(embedded_dataset[0]))

60000
2


In [10]:
print(len(emb_model.encode("0")))

384


In [11]:
print(len(embedded_dataset[0][0]))

64


In [12]:
print(len(embedded_dataset[0][1]))

384


In [13]:
cINN = ConditionalRealNVP(64,128,20,384)

In [14]:
encoded_imgs, embedded_labels = zip(*embedded_dataset)

In [20]:
train_cinn(cINN, encoded_imgs, embedded_labels, batch_size=128)

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.